In [1]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

from federated_utils_fedavg_copy import *

 
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [3]:
#declear path to your data
drebin_data_path = r'data\drebin.csv'
malgenome_data_path = r'data\malgenome.csv'
kronodroid_data_path = r'data\kronodroid.csv'
TUANDROMD_data_path=r'data\TUANDROMD.csv'



Drebin_data = pd.read_csv(drebin_data_path, header = None)

Malgenome_data = pd.read_csv(malgenome_data_path)

Tuandromd_data=pd.read_csv(TUANDROMD_data_path)

kronodroid_data=pd.read_csv(kronodroid_data_path)
Kronodroid_data = kronodroid_data.iloc[:,range(1,kronodroid_data.shape[1])]




## fedavg

In [7]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

def train_model(model, train_loader, loss_fn, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']
for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients(X_train, y_train, num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedAvg 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedAvg 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]

                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)


                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    train_model(local_model, train_loader, loss, optimizer)


                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()
                
                # ...
                
                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedAvg-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedAvg-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedAvg-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 10
No. of Rounds: 100
---------------------------------------------
|=======================|
|Traditional FedAvg 2017|
|=======================|
comm_round: 0 | global_acc: 63.132% | global_loss: 0.6683652400970459 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.7163834033478775| flobal_FPR: 1.0 
comm_round: 1 | global_acc: 63.132% | global_loss: 0.6562666296958923 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.781741707348227| flobal_FPR: 1.0 
comm_round: 2 | global_acc: 63.132% | global_loss: 0.6440979838371277 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.8235436428740673| flobal_FPR: 1.0 
comm_round: 3 | global_acc: 63.132% | global_loss: 0.6308736205101013 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.8546983344183332| flobal_FPR: 1.0 
comm_round: 4 | global_acc: 63.132% | gl

## fed avg non iid

In [6]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

def train_model(model, train_loader, loss_fn, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']
for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients_non_iid(X_train, [tuple(label) for label in y_train.astype(int).tolist()], num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedAvg 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedAvg non idd 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]

                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)


                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    train_model(local_model, train_loader, loss, optimizer)


                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()
                
                # ...
                
                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedAvg-noniid-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedAvg-noniid-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedAvg-noniid-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 10
No. of Rounds: 100
---------------------------------------------
Client client_1: {(0,): 744, (1,): 459}
Client client_2: {(0,): 725, (1,): 478}
Client client_3: {(0,): 130, (1,): 1073}
Client client_4: {(0,): 627, (1,): 576}
Client client_5: {(0,): 568, (1,): 635}
Client client_6: {(0,): 704, (1,): 499}
Client client_7: {(0,): 574, (1,): 629}
Client client_8: {(0,): 311, (1,): 892}
Client client_9: {(0,): 759, (1,): 444}
Client client_10: {(0,): 345, (1,): 858}
|=======================|
|Traditional FedAvg non idd 2017|
|=======================|
comm_round: 0 | global_acc: 36.902% | global_loss: 0.7078294157981873 | global_f1: 0.5388726919339164 | global_precision: 0.36880611905553706 | global_recall: 1.0 | global_auc: 0.633130910815858| flobal_FPR: 0.0 
comm_round: 1 | global_acc: 36.902% | global_loss: 0.706152081489563 | global_f1: 0.5388726919339164 | global_precision: 0.36880611905553706 | globa